In [1]:
import sys 
import os
main_dir = '/ocean/projects/asc170022p/singla/Explanation_XRay'
sys.path.append(main_dir)
from classifier.DenseNet import pretrained_classifier
from unet.SegmentationNetwork import UNET
from explainer.networks_128 import Discriminator, Discriminator_Ordinal, Generator_Encoder_Decoder
import tensorflow.contrib.slim as slim
import tensorflow as tf
import numpy as np
from utils import *
from losses import *
import pdb
import yaml
import time
import scipy.io as sio
from datetime import datetime
import random
import warnings
import argparse
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
input_size = 256
NUMS_CLASS = 10
NUMS_CLASS_cls = 14
channels=1
batch_size = 8

In [3]:
with tf.name_scope('some_scope1'):
    x_source = tf.placeholder(tf.float32, [batch_size, input_size, input_size, channels])
    train_phase = tf.placeholder(tf.bool)
    y_s = tf.placeholder(tf.int32, [batch_size, NUMS_CLASS])  
    y_source = y_s[:,0]
    y_t = tf.placeholder(tf.int32, [batch_size, NUMS_CLASS]) 
    y_target = y_t[:,0]
    # ============= G & D =============    
    G = Generator_Encoder_Decoder("generator") # with conditional BN, SAGAN: SN here as well
    D = Discriminator_Ordinal("discriminator") #with SN and projection 
    U = UNET()

    real_source_logits = D(x_source, y_s, NUMS_CLASS, "NO_OPS")
    fake_target_img, fake_target_img_embedding = G(x_source, train_phase, y_target, NUMS_CLASS, channels)
    fake_source_img, fake_source_img_embedding = G(fake_target_img, train_phase, y_source, NUMS_CLASS, channels)
    fake_source_recons_img, x_source_img_embedding = G(x_source, train_phase, y_source, NUMS_CLASS, channels)    
    fake_target_logits = D(fake_target_img, y_t, NUMS_CLASS, None)    
    # ============= pre-trained classifier =============      
    real_img_cls_logit_pretrained,  real_img_cls_prediction = pretrained_classifier(x_source, NUMS_CLASS_cls, reuse=False, name='classifier')
    fake_img_cls_logit_pretrained, fake_img_cls_prediction = pretrained_classifier(fake_target_img, NUMS_CLASS_cls, reuse=True)
    real_img_recons_cls_logit_pretrained, real_img_recons_cls_prediction = pretrained_classifier(fake_source_img, NUMS_CLASS_cls, reuse=True)
    # ============= pre-trained segmentation =============    
    _, seg_x_source = U(x_source)



Tensor("some_scope1/Placeholder:0", shape=(8, 256, 256, 1), dtype=float32)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Tensor("some_scope1/discriminator/add:0", shape=(8, 128, 128, 64), dtype=float32)
Tensor("some_scope1/discriminator/add_1:0", shape=(8, 64, 64, 128), dtype=float32)
Tensor("some_scope1/discriminator/add_2:0", shape=(8, 32, 32, 256), dtype=float32)
Tensor("some_scope1/discriminator/add_3:0", shape=(8, 16, 16, 512), dtype=float32)
Tensor("some_scope1/discriminator/add_4:0", shape=(8, 8, 8, 1024), dtype=float32)
Tensor("some_scope1/discriminator/add_5:0", shape=(8, 8, 8, 1024), dtype=float32)
Tensor("some_scope1/discriminator/Relu:0", shape=(8, 8, 8, 1024), dtype=float32)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Encoder-Decoder
Tensor("some_scope1/Placeholder:0", shape=(8, 256, 256, 1), dtype=float32)
(':', <tf.Tensor 'some_scope1/generator/BiasAdd:0' shape=(8, 25

False
('Classifier', False)
('inputae: ', <tf.Tensor 'some_scope1/Tanh:0' shape=(8, 256, 256, 1) dtype=float32>)
('bn final: ', <tf.Tensor 'some_scope1/classifier_1/linear_batch/FusedBatchNorm:0' shape=(8, 8, 8, 1024) dtype=float32>)
False
('Classifier', False)
('inputae: ', <tf.Tensor 'some_scope1/Tanh_1:0' shape=(8, 256, 256, 1) dtype=float32>)
('bn final: ', <tf.Tensor 'some_scope1/classifier_2/linear_batch/FusedBatchNorm:0' shape=(8, 8, 8, 1024) dtype=float32>)
False
('x: ', <tf.Tensor 'some_scope1/Placeholder:0' shape=(8, 256, 256, 1) dtype=float32>)
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
('conv1: ', <tf.Tensor 'some_scope1/UNet/conv2d_1/Relu:0' shape=(8, 256, 256, 32) dtype=float32>)
('pool1: ', <tf.Tensor 'some_scope1/UNet/max_pooling2d/MaxPool:0' shape=(8, 128, 128, 32) dtype=float32>)
('conv2: ', <tf.Tensor 'some_scope1/UNet/conv2d_3/Relu:0' shape=(8, 128, 128, 64) dtype=float32>)
('pool2: ', <tf.Tensor 'some_scope1/UNet/max_pooling2d_1/MaxPool:0' sha

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver() 

In [11]:
# Print model size. 
vars = 0
for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
    vars += np.prod(v.get_shape().as_list())
print(vars*4)/(1024**2),"MB"

460 MB


In [6]:
all_tensors = [tensor for op in tf.get_default_graph().get_operations() for tensor in op.values()]
for v in all_tensors:
    if v.get_shape() == None:
        continue
    vars += np.prod(v.get_shape().as_list())
print(vars*4)/(1024**2),"MB"

41866.82236480713 MB


In [9]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='some_scope1')

[]

In [9]:
tf.global_variables()

[<tf.Variable 'discriminator/ResBlock1/conv1/W:0' shape=(3, 3, 1, 64) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock1/conv1/b:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock1/conv1/snu:0' shape=(1, 64) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock1/conv2/W:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock1/conv2/b:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock1/conv2/snu:0' shape=(1, 64) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock1/identity/W:0' shape=(1, 1, 1, 64) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock1/identity/b:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock1/identity/snu:0' shape=(1, 64) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock2/conv1/W:0' shape=(3, 3, 64, 128) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock2/conv1/b:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'discriminator/ResBlock2/co

In [15]:
[(n.name, n.size) for n in tf.get_default_graph().as_graph_def().node]

AttributeError: size

In [24]:
all_tensors[0].get_shape().as_list()

[None, 256, 256, 1]

In [25]:
np.prod(all_tensors[0].get_shape().as_list())

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'